In [41]:
# 构建语料库，每行包括中文、英文（解码器输入）和翻译成英文后的目标输出3共和句子
sentences = [
    ['咖哥 喜欢 小冰', '<sos> KaGe likes XiaoBing', 'KaGe likes XiaoBing <eos>'],
    ['我 爱 学习 人工智能', '<sos> I love studying AI', 'I love studying AI <eos>'],
    ['深度学习 改变 世界', '<sos> DL changed the world', 'DL changed the world <eos>'],
    ['自然 语言 处理 很 强大', '<sos> NLP is so powerful', 'NLP is so powerful <eos>'],
    ['神经网络 非常 复杂', '<sos> Neural-Nets are complex', 'Neural-Nets are complex <eos>'],
]
word_list_cn, word_list_en = [], []  # 中英文词汇表
# 遍历每一个句子并将单词添加到对应的词汇表中
for cn, en, en2 in sentences:
    word_list_cn.extend(cn.split())
    word_list_en.extend(en.split())
    word_list_en.extend(en2.split())
# 去重
word_list_cn = list(set(word_list_cn))
word_list_en = list(set(word_list_en))
# 构建单词到索引的映射
word2idx_cn = {w: i for i, w in enumerate(word_list_cn)}
word2idx_en = {w: i for i, w in enumerate(word_list_en)}
# 构建索引到单词的映射
idx2word_cn = {i: w for i, w in enumerate(word_list_cn)}
idx2word_en = {i: w for i, w in enumerate(word_list_en)}
# 计算词汇表的大小
vocab_size_cn = len(word_list_cn)
vocab_size_en = len(word_list_en)
print("句子数量: ", len(sentences))
print("中文词汇表大小: ", vocab_size_cn)
print("英文词汇表大小: ", vocab_size_en)
print("中文词汇表: ", word2idx_cn)
print("英文词汇表: ", word2idx_en)

句子数量:  5
中文词汇表大小:  18
英文词汇表大小:  20
中文词汇表:  {'爱': 0, '非常': 1, '语言': 2, '深度学习': 3, '强大': 4, '很': 5, '喜欢': 6, '我': 7, '神经网络': 8, '改变': 9, '复杂': 10, '世界': 11, '处理': 12, '小冰': 13, '人工智能': 14, '自然': 15, '学习': 16, '咖哥': 17}
英文词汇表:  {'Neural-Nets': 0, 'love': 1, 'I': 2, 'world': 3, 'AI': 4, 'complex': 5, 'is': 6, '<eos>': 7, 'likes': 8, 'DL': 9, 'NLP': 10, 'the': 11, 'XiaoBing': 12, 'powerful': 13, '<sos>': 14, 'changed': 15, 'KaGe': 16, 'studying': 17, 'so': 18, 'are': 19}


In [42]:
import numpy as np
import torch
import random
# 定义一个函数，随机选择一个句子和词汇表表示生成输入，输出和目标数据
def make_data(sentences):
    # 随机选择一个句子进行训练
    sentence = random.choice(sentences)
    # 将输入句子中的单词转化为对应的索引
    encoder_input = np.array([[word2idx_cn[w] for w in sentence[0].split()]])
    # 将输出句子中的单词转化为对应的索引
    decoder_input = np.array([[word2idx_en[w] for w in sentence[1].split()]])
    # 将目标句子中的单词转化为对应的索引
    target = np.array([[word2idx_en[w] for w in sentence[2].split()]])
    # 将输入、输出、目标批次转化为LongTensor
    encoder_input = torch.LongTensor(encoder_input)
    decoder_input = torch.LongTensor(decoder_input)
    target = torch.LongTensor(target)
    return encoder_input, decoder_input, target
# 使用make_data函数生成输入、输出和目标张量
encoder_input, decoder_input, target = make_data(sentences)
original_sentence = None
for s in sentences:  # 获取原始句子
    if all([word2idx_cn[w] in encoder_input[0] for w in s[0].split()]):
        original_sentence = s
        break
print("原始句子: ", original_sentence)
print("编码器输入张量的形状：", encoder_input.shape)
print("解码器输入张量的形状：", decoder_input.shape)
print("目标张量的形状：", target.shape)
print("编码器输入张量：", encoder_input)
print("解码器输入张量：", decoder_input)
print("目标张量：", target)

原始句子:  ['我 爱 学习 人工智能', '<sos> I love studying AI', 'I love studying AI <eos>']
编码器输入张量的形状： torch.Size([1, 4])
解码器输入张量的形状： torch.Size([1, 5])
目标张量的形状： torch.Size([1, 5])
编码器输入张量： tensor([[ 7,  0, 16, 14]])
解码器输入张量： tensor([[14,  2,  1, 17,  4]])
目标张量： tensor([[ 2,  1, 17,  4,  7]])


In [43]:
import torch.nn as nn
# 定义编码器类和解码器类
class Encoder(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(Encoder, self).__init__()
        self.hidden_size = hidden_size  # 隐藏层大小
        self.embedding = nn.Embedding(input_size, hidden_size)  # 词嵌入层
        self.rnn = nn.RNN(hidden_size, hidden_size, batch_first=True)  # RNN层
    def forward(self, input, hidden):
        embedded = self.embedding(input)  # 词嵌入
        output, hidden = self.rnn(embedded, hidden)
        return output, hidden
# 定义解码器类
class Decoder(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(Decoder, self).__init__()
        self.hidden_size = hidden_size  # 隐藏层大小
        self.embedding = nn.Embedding(output_size, hidden_size)  # 词嵌入层
        self.rnn = nn.RNN(hidden_size, hidden_size, batch_first=True)  # RNN层
        self.out = nn.Linear(hidden_size, output_size)  # 输出层
    def forward(self, input, hidden):
        embedded = self.embedding(input)  # 词嵌入
        output, hidden = self.rnn(embedded, hidden)
        output = self.out(output)  # 输出
        return output, hidden
n_hidden = 128  # 隐藏层大小
# 创建编码器和解码器
encoder = Encoder(vocab_size_cn, n_hidden)
decoder = Decoder(n_hidden, vocab_size_en)
print("编码器结构：", encoder)
print("解码器结构：", decoder)

编码器结构： Encoder(
  (embedding): Embedding(18, 128)
  (rnn): RNN(128, 128, batch_first=True)
)
解码器结构： Decoder(
  (embedding): Embedding(20, 128)
  (rnn): RNN(128, 128, batch_first=True)
  (out): Linear(in_features=128, out_features=20, bias=True)
)


In [44]:
# 组合编码器和解码器
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder):
        super(Seq2Seq, self).__init__()
        self.encoder = encoder  # 编码器
        self.decoder = decoder  # 解码器
    def forward(self, encoder_input, hidden, decoder_input):
        # 使输入序列通过编码器冰获取输出和隐藏状态
        encoder_output, encoder_hidden = self.encoder(encoder_input, None)
        # 将编码器的隐藏状态作为解码器的初始隐藏状态
        decoder_hidden = encoder_hidden
        # 使解码器的输入序列通过解码器并获取输出
        decoder_output, _ = self.decoder(decoder_input, decoder_hidden)
        return decoder_output
# 创建Seq2Seq模型
model = Seq2Seq(encoder, decoder)
print("Seq2Seq模型结构：", model)

Seq2Seq模型结构： Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(18, 128)
    (rnn): RNN(128, 128, batch_first=True)
  )
  (decoder): Decoder(
    (embedding): Embedding(20, 128)
    (rnn): RNN(128, 128, batch_first=True)
    (out): Linear(in_features=128, out_features=20, bias=True)
  )
)


In [45]:
# 定义训练函数
def train_seq2seq(model, criterion, optimizer, epochs):
    for epoch in range(epochs):
        encoder_input, decoder_input, target = make_data(sentences)  # 生成输入、输出和目标数据
        hidden = torch.zeros(1, encoder_input.size(0), n_hidden)  # 初始化隐藏状态
        optimizer.zero_grad()
        output = model(encoder_input, hidden, decoder_input)
        loss = criterion(output.view(-1, vocab_size_en), target.view(-1))
        if (epoch + 1) % 100 == 0:
            print("Epoch: %d, Loss: %1.04f" % (epoch + 1, loss.item()))
        loss.backward()
        optimizer.step()
# 训练模型
epochs = 400
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
train_seq2seq(model, criterion, optimizer, epochs)

Epoch: 100, Loss: 0.0740
Epoch: 200, Loss: 0.0184
Epoch: 300, Loss: 0.0096
Epoch: 400, Loss: 0.0064


In [50]:
# 定义预测函数
def test_seq2seq(model, source_sentences):
    # 将输入句子转化为索引
    encoder_input = np.array([[word2idx_cn[w] for w in source_sentences.split()]])
    # 构建输出句子的索引，以'<sos>'开始，后面跟'<eos>'，长度与输入句子相同
    decoder_input = np.array([[word2idx_en['<sos>']] + [word2idx_en['<eos>']] * (len(encoder_input[0]) - 1)])
    # 将输入、输出转化为LongTensor
    encoder_input = torch.LongTensor(encoder_input)
    decoder_input = torch.LongTensor(decoder_input)
    hidden = torch.zeros(1, encoder_input.size(0), n_hidden)  # 初始化隐藏状态
    predicted = model(encoder_input, hidden, decoder_input)  # 预测
    predicted = predicted.data.max(2, keepdim=True)[1]  # 获取预测结果
    # 打印输入结果何预测结果
    print(source_sentences, '->', ' '.join([idx2word_en[i.item()] for i in predicted.squeeze()]))
# 测试模型
test_seq2seq(model, '咖哥 喜欢 小冰')
test_seq2seq(model, "自然 语言 处理 很 强大")
test_seq2seq(model, '我 爱 学习 人工智能')
test_seq2seq(model, '小冰 喜欢 咖哥')

咖哥 喜欢 小冰 -> KaGe likes XiaoBing
自然 语言 处理 很 强大 -> NLP is so powerful <eos>
我 爱 学习 人工智能 -> I love studying <eos>
小冰 喜欢 咖哥 -> KaGe likes XiaoBing
